In [ ]:
import os
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.exceptions import UnexpectedResponse
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

/home/shivampr21/codegen/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# FastEmbedEmbeddings()

In [18]:
# Step 1: Configure paths and settings
REPO_PATH = "./"  # Current repository path
COLLECTION_NAME = "code-repository"
OLLAMA_MODEL = "gemma3:12b"  # You can change to any model available in your Ollama instance

In [4]:
# Step 2: Set up the embedding model
embeddings = FastEmbedEmbeddings(model_name="thenlper/gte-large")

/home/shivampr21/codegen/.venv/lib/python3.13/site-packages/langchain_community/embeddings/fastembed.py:109: UserWarning: The model thenlper/gte-large now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  values["model"] = fastembed.TextEmbedding(
Fetching 5 files: 100%|██████████| 5/5 [03:31<00:00, 42.38s/it] 


In [5]:
# Step 3: Initialize Qdrant client
client = QdrantClient(path="./qdrant_db")  # Store locally

In [6]:
sample_text = "S"
sample_embedding = embeddings.embed_query(sample_text)

In [7]:
# embeddings.

In [8]:
# Step 4: Create or get collection
try:
    collection_info = client.get_collection(collection_name=COLLECTION_NAME)
    print(f"Collection '{COLLECTION_NAME}' already exists")
except (UnexpectedResponse, ValueError):
    # Create new collection with cosine similarity
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=(len(embeddings.embed_query("Nishtha Pandey"))), distance=Distance.COSINE),
    )
    print(f"Created new collection: '{COLLECTION_NAME}'")

Collection 'code-repository' already exists


In [9]:
# Step 5: Create vector store
vector_store = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings,
)

In [10]:
# Step 6: Load and process code repository
def load_code_repository(repo_path):
    # Define file extensions to include
    code_extensions = [".py", ".js", ".java", ".cpp", ".c", ".h", ".cs", ".go", ".rb", ".php", ".ts", ".html", ".css"]

    # Create a list of glob patterns for each extension
    glob_patterns = [f"**/*{ext}" for ext in code_extensions]

    # Load all code files
    all_files = []
    for pattern in glob_patterns:
        loader = DirectoryLoader(
            repo_path,
            glob=pattern,
            loader_cls=TextLoader,
            show_progress=True
        )
        files = loader.load()
        all_files.extend(files)

    print(f"Loaded {len(all_files)} code files from repository")
    return all_files

In [11]:
# Step 7: Split documents into chunks
def process_documents(documents):
    # Use a code-optimized splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=150,
        separators=["\nclass ", "\ndef ", "\n\n", "\n", " ", ""]
    )

    chunks = text_splitter.split_documents(documents)
    print(f"Created {len(chunks)} chunks from code repository")
    return chunks

In [12]:
# Step 8: Index the repository
def index_repository():
    documents = load_code_repository(REPO_PATH)
    chunks = process_documents(documents)

    # Add documents to vector store
    vector_store.add_documents(documents=chunks)
    print("Repository indexed successfully")

In [24]:
# Step 9: Set up the LLM with Ollama
llm = OllamaLLM(model=OLLAMA_MODEL, base_url="http://100.98.113.21:11434")

In [25]:
# Step 10: Create a custom prompt template for code-related queries
prompt_template = """
You are an expert software developer assistant. Use the following pieces of context from the code repository to answer the question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [26]:
# Step 11: Create the RAG chain
def create_rag_chain():
    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}  # Retrieve top 5 most relevant chunks
    )

    # Create the RAG chain
    rag_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    return rag_chain

In [27]:
# Step 12: Query function
def query_code_repository(query, rag_chain=None):
    if rag_chain is None:
        rag_chain = create_rag_chain()

    result = rag_chain({"query": query})

    return {
        "answer": result["result"],
        "source_documents": result.get("source_documents", [])
    }

In [30]:
# # Main execution
# if __name__ == "__main__":
#     # Index the repository (only need to run this once or when code changes)
#     index_repository()

#     # Create the RAG chain
#     rag_chain = create_rag_chain()

#     # Example usage
#     # while True:
#     query = input("\nEnter your code question (or 'exit' to quit): ")
#     # if query.lower() == 'exit':
#     #     break

#     result = query_code_repository(query, rag_chain)

#     print("\nAnswer:")
#     print(result["answer"])

#     print("\nSources:")
#     for i, doc in enumerate(result["source_documents"]):
#         print(f"\nSource {i+1}:")
#         print(f"Path: {doc.metadata.get('source', 'Unknown')}")
#         print(f"Content: {doc.page_content[:150]}...")

In [29]:
import gradio as gr
# from langchain_community.llms import Ollama
# from langchain.chains import RetrievalQA

# Reuse the RAG chain from the previous code
rag_chain = create_rag_chain()

def process_query(query):
    if not query.strip():
        return "Please enter a valid query."

    result = query_code_repository(query, rag_chain)

    # Format the response
    response = result["answer"]

    # Add source information
    response += "\n\n**Sources:**\n"
    for i, doc in enumerate(result["source_documents"]):
        source_path = doc.metadata.get('source', 'Unknown')
        response += f"\n{i+1}. {source_path}"

    return response

# Create the Gradio interface
demo = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about your code repository..."),
    outputs="markdown",
    title="Local Code Repository RAG System",
    description="Ask questions about your code repository and get context-aware answers."
)

# Launch the interface
demo.launch(share=False)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
